In [2]:
"""
A freely-propagating, premixed hydrogen flat flame with multicomponent
transport properties.
"""

import cantera as ct
import numpy as np

# Simulation parameters
p = ct.one_atm  # pressure [Pa]
Tin = 300.0  # unburned gas temperature [K]
reactants = 'H2:1.1, O2:1, AR:5'  # premixed gas composition
width = 0.03  # m
loglevel = 1  # amount of diagnostic output (0 to 8)

# IdealGasMix object used to compute mixture properties, set to the state of the
# upstream fuel-air mixture
gas = ct.Solution('h2o2.xml')
gas.TPX = Tin, p, reactants

# Set up flame object
f = ct.FreeFlame(gas, width=width)
f.set_refine_criteria(ratio=3, slope=0.06, curve=0.12)
f.show_solution()

# Solve with mixture-averaged transport model
f.transport_model = 'Mix'
f.solve(loglevel=loglevel, auto=True)
f.show_solution()




>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> reactants <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    Mass Flux:        1.339 kg/m^2/s 
    Temperature:        300 K 
    Mass Fractions: 
                      H2    0.009478 
                      O2      0.1368 
                      AR      0.8537 



>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> flame <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    Pressure:   1.013e+05 Pa

-------------------------------------------------------------------------------
          z           u           V           T      lambda      eField 
-------------------------------------------------------------------------------
          0           1           0         300           0           0 
      0.006           1           0         300           0           0 
      0.009           1           0         300           0           0 
     0.0105       2.206           0       698.2           0           0 
      0.012       3.411           0        1096           0           0 
      0.0

Problem solved on [110] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 
    to resolve H H2O2 HO2 
##############################################################################

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [134] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 32 33 34 35 36 37 
    to resolve HO2 
##############################################################################

..............................

    0.02363   3.067e-05   3.626e-06   0.0002345      0.0633    0.001329 
      0.024    2.45e-05   2.579e-06   0.0001875     0.06334    0.001197 
    0.02438   2.002e-05   1.901e-06   0.0001534     0.06338    0.001089 
    0.02475   1.669e-05   1.446e-06   0.0001281     0.06341   0.0009994 
    0.02513   1.416e-05    1.13e-06   0.0001089     0.06344   0.0009246 
     0.0255   1.222e-05   9.059e-07   9.412e-05     0.06347   0.0008618 
    0.02625    9.62e-06   6.339e-07   7.434e-05     0.06351   0.0007687 
      0.027   7.879e-06   4.709e-07   6.107e-05     0.06354   0.0006986 
     0.0285   5.971e-06    3.12e-07   4.649e-05     0.06357   0.0006114 
       0.03   4.888e-06   2.322e-07    3.82e-05      0.0636   0.0005553 
      0.033   3.945e-06   1.693e-07   3.096e-05     0.06362   0.0005008 
      0.036   3.515e-06    1.43e-07   2.765e-05     0.06364   0.0004737 
      0.042   3.244e-06   1.271e-07   2.556e-05     0.06365   0.0004556 
      0.048   3.157e-06   1.221e-07   2.489e-05    

In [ ]:
# Solve with the energy equation enabled
f.save('h2_adiabatic.xml', 'mix', 'solution with mixture-averaged transport')
f.show_solution()
print('mixture-averaged flamespeed = {0:7f} m/s'.format(f.u[0]))

# Solve with multi-component transport properties
f.transport_model = 'Multi'
f.solve(loglevel) # don't use 'auto' on subsequent solves
f.show_solution()
print('multicomponent flamespeed = {0:7f} m/s'.format(f.u[0]))
f.save('h2_adiabatic.xml','multi', 'solution with multicomponent transport')

# write the velocity, temperature, density, and mole fractions to a CSV file
f.write_csv('h2_adiabatic.csv', quiet=False)